# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor
import numpy as np

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f92141a4940>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../../input_files/ap2.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()
vec_rn = VectorizedRxnNet(rn, dev='cpu')

['default_assoc', 1.0]
['A']
['M']
['B']
['S']
Parsing rule...
['A(a)+B(b)<->A(a!1).B(a!1)', '', 'G=-20']
['A', 'B', '']
['A(a)+B(b)', 'A(a!1).B(a!1)']
GGGGGGGGGgg
Parsing rule...
['A(b)+M(a)<->A(b!1).M(a!1)', '', 'G=-20']
['A', 'M', '']
['A(b)+M(a)', 'A(b!1).M(a!1)']
GGGGGGGGGgg
Parsing rule...
['A(c)+S(a)<->A(c!1).S(a!1)', '', 'G=-20']
['A', 'S', '']
['A(c)+S(a)', 'A(c!1).S(a!1)']
GGGGGGGGGgg
Parsing rule...
['B(b)+M(b)<->B(b!1).M(b!1)', '', 'G=-20']
['B', 'M', '']
['B(b)+M(b)', 'B(b!1).M(b!1)']
GGGGGGGGGgg
Parsing rule...
['M(c)+S(b)<->M(c!1).S(b!1)', '', 'G=-20']
['M', 'S', '']
['M(c)+S(b)', 'M(c!1).S(b!1)']
GGGGGGGGGgg
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f917d3b6940>, 'copies': tensor([100.], dtype=torch.float64)})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f917d3b6940>, 'copies': tensor([100.], dtype=torch.float64)})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph obj

{'S', 'A'}
{'B'}
Steric hindrance detected
Node-1 :  (5, {'struct': <networkx.classes.graph.Graph object at 0x7f917d1f0f28>, 'copies': tensor([0.], dtype=torch.float64)})
Node-2 :  (7, {'struct': <networkx.classes.graph.Graph object at 0x7f917d215208>, 'copies': tensor([0.], dtype=torch.float64)})
-----
{'B', 'A'}
{'M', 'B'}
{'A'}
Steric hindrance detected
Node-1 :  (5, {'struct': <networkx.classes.graph.Graph object at 0x7f917d1f0f28>, 'copies': tensor([0.], dtype=torch.float64)})
Node-2 :  (8, {'struct': <networkx.classes.graph.Graph object at 0x7f917d215eb8>, 'copies': tensor([0.], dtype=torch.float64)})
-----
{'B', 'A'}
{'M', 'S'}
{'B', 'A'}
False
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'M')
############################3
Allowed edges: 
('A', 'S')
############################3
Allowed edges: 
('B', 'M')
############################3
Allowed edges: 
('M', 'S')
[(13, {'struct': <networkx.classes.graph.Graph object at 0x7f917d1aa4e0>, 'copies': tensor([0.], dtype=torch.float6

Node-1 :  (10, {'struct': <networkx.classes.graph.Graph object at 0x7f917ea81978>, 'copies': tensor([0.], dtype=torch.float64)})
Node-2 :  (9, {'struct': <networkx.classes.graph.Graph object at 0x7f917d2152e8>, 'copies': tensor([0.], dtype=torch.float64)})
-----
{'M', 'S', 'A'}
{'M', 'B', 'A'}
{'S'}
Steric hindrance detected
Node-1 :  (10, {'struct': <networkx.classes.graph.Graph object at 0x7f917ea81978>, 'copies': tensor([0.], dtype=torch.float64)})
Node-2 :  (10, {'struct': <networkx.classes.graph.Graph object at 0x7f917ea81978>, 'copies': tensor([0.], dtype=torch.float64)})
-----
{'M', 'S', 'A'}
{'M', 'S', 'A'}
set()
Steric hindrance detected
Node-1 :  (10, {'struct': <networkx.classes.graph.Graph object at 0x7f917ea81978>, 'copies': tensor([0.], dtype=torch.float64)})
Node-2 :  (11, {'struct': <networkx.classes.graph.Graph object at 0x7f917d1aa978>, 'copies': tensor([0.], dtype=torch.float64)})
-----
{'M', 'S', 'A'}
{'B', 'S', 'A'}
{'M'}
Steric hindrance detected
Node-1 :  (10, {'

In [4]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

{(0, 4): 0, (0, 5): 1, (0, 6): 2, (0, 9): 17, (0, 10): 18, (0, 13): 20, (1, 4): 0, (1, 7): 3, (1, 8): 4, (1, 9): 5, (1, 10): 6, (1, 13): 19, (2, 5): 1, (2, 7): 3, (2, 9): 7, (2, 11): 8, (2, 12): 9, (2, 13): 10, (3, 6): 2, (3, 8): 4, (3, 10): 11, (3, 11): 12, (3, 12): 13, (3, 13): 14, (4, 9): 7, (4, 10): 11, (5, 9): 5, (5, 11): 12, (5, 13): 15, (6, 10): 6, (6, 11): 8, (6, 13): 16, (7, 12): 13, (7, 13): 16, (7, 9): 17, (8, 12): 9, (8, 13): 15, (8, 10): 18, (9, 13): 14, (10, 13): 10, (11, 13): 19, (12, 13): 20}


In [5]:
#Do modifications here
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([1.]*np.array(1e0))
update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
vec_rn = VectorizedRxnNet(rn, dev='cpu')
print(vec_rn.kon)

#Changing initial concentrations

{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 17}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 18}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-60.], dtype=torch.float64), 'uid': 20}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 4}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 5}
{'k_on'

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [6]:
# vec_rn.update_reaction_net(rn)
# poly_system = EquilibriumSolver(rn)
# solution = poly_system.solve()
# print(solution)
# if solution == None:
#     print("No Equilibrium solution")
# else:
#     print(solution)
#     print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
# print(vec_rn.kon)

In [7]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

[['A', 'AM', 'ABM', 'ABMS'], ['M', 'AM', 'ABM', 'ABMS'], ['B', 'AB', 'ABM', 'ABMS'], ['S', 'AS', 'AMS', 'ABMS'], ['AM', 'ABM', 'ABMS'], ['AB', 'ABM', 'ABMS'], ['AS', 'AMS', 'ABMS'], ['BM', 'BMS', 'ABMS'], ['MS', 'BMS', 'ABMS'], ['ABM', 'ABMS'], ['AMS', 'ABMS'], ['ABS', 'ABMS'], ['BMS', 'ABMS'], ['ABMS']]
[3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0]


In [8]:
# if solution != None:
# #     for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
#         print(k," : ", v.item())

In [9]:
# if solution !=None:
#     for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
#         print(k," : ", v)

## Using the optimizer with a 1 second simulation runtime ##

In [ ]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=1000,
                  learning_rate=new_kon[0].item()/1e2,
                  device='cpu',method="RMSprop")
optim.rn.update_reaction_net(rn)
optim.optimize(optim='flux_coeff',corr_rxns=[[10],[0]])

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.], dtype=torch.float64, requires_grad=True)]
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 0 was -0.5557468533726071
current params: tensor([1.1000, 1.1000, 1.1000, 1.1000, 1.1000, 1.0000, 1.0000, 1.0000, 1.1000,
        1.1000, 1.0000, 1.0000, 1.1000, 1.1000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 1 was -0.5633655315046345
current params: tensor([1.1709, 1.1709, 1.1709, 1.1709, 1.1709, 1.0001, 1.0001, 1.0001, 1.1709,
        1.1709, 1.0001, 1.0001, 1.1709, 1.1709, 1.0001, 1.0000, 1.0000, 1.0001,
        1.0001, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 2 was -0.5625205123762335
current params: tensor

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 25 was -0.5827661730244666
current params: tensor([1.9042, 1.9042, 1.9042, 1.9042, 1.9042, 1.0011, 1.0011, 1.0011, 1.9042,
        1.9042, 1.0011, 1.0011, 1.9042, 1.9042, 1.0011, 1.0000, 1.0000, 1.0011,
        1.0011, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 26 was -0.5743147633546163
current params: tensor([1.9247, 1.9247, 1.9247, 1.9247, 1.9247, 1.0011, 1.0011, 1.0011, 1.9247,
        1.9247, 1.0011, 1.0011, 1.9247, 1.9247, 1.0011, 1.0000, 1.0000, 1.0011,
        1.0011, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 27 was -0.5736845106900789
current params: tensor([1.9449, 1.9449, 1.9449, 1.9449, 1.9449, 1.0012, 1.0012, 1.0012, 1.9449,
        1.9449, 1.0012, 1.0012, 1.9449, 1.9449, 1.0012, 1.0000, 1.0000, 1.0012,
        1.0012, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], 

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 50 was -0.5767297936703555
current params: tensor([2.3484, 2.3484, 2.3484, 2.3484, 2.3484, 1.0022, 1.0022, 1.0022, 2.3484,
        2.3484, 1.0022, 1.0022, 2.3484, 2.3484, 1.0022, 1.0000, 1.0000, 1.0022,
        1.0022, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 51 was -0.5762674089486282
current params: tensor([2.3641, 2.3641, 2.3641, 2.3641, 2.3641, 1.0022, 1.0022, 1.0022, 2.3641,
        2.3641, 1.0022, 1.0022, 2.3641, 2.3641, 1.0022, 1.0000, 1.0000, 1.0022,
        1.0022, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 52 was -0.5758105819224999
current params: tensor([2.3797, 2.3797, 2.3797, 2.3797, 2.3797, 1.0022, 1.0022, 1.0022, 2.3797,
        2.3797, 1.0022, 1.0022, 2.3797, 2.3797, 1.0022, 1.0000, 1.0000, 1.0022,
        1.0022, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], 

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 75 was -0.5665594667948535
current params: tensor([2.7128, 2.7128, 2.7128, 2.7128, 2.7128, 1.0032, 1.0032, 1.0032, 2.7128,
        2.7128, 1.0032, 1.0032, 2.7128, 2.7128, 1.0032, 1.0000, 1.0000, 1.0032,
        1.0032, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 76 was -0.566202384668598
current params: tensor([2.7264, 2.7264, 2.7264, 2.7264, 2.7264, 1.0033, 1.0033, 1.0033, 2.7264,
        2.7264, 1.0033, 1.0033, 2.7264, 2.7264, 1.0033, 1.0000, 1.0000, 1.0033,
        1.0033, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 77 was -0.5658484414731023
current params: tensor([2.7400, 2.7400, 2.7400, 2.7400, 2.7400, 1.0033, 1.0033, 1.0033, 2.7400,
        2.7400, 1.0033, 1.0033, 2.7400, 2.7400, 1.0033, 1.0000, 1.0000, 1.0033,
        1.0033, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 100 was -0.565777765923652
current params: tensor([3.0386, 3.0386, 3.0386, 3.0386, 3.0386, 1.0043, 1.0043, 1.0043, 3.0386,
        3.0386, 1.0043, 1.0043, 3.0386, 3.0386, 1.0043, 1.0000, 1.0000, 1.0043,
        1.0043, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 101 was -0.5654783261231461
current params: tensor([3.0511, 3.0511, 3.0511, 3.0511, 3.0511, 1.0043, 1.0043, 1.0043, 3.0511,
        3.0511, 1.0043, 1.0043, 3.0511, 3.0511, 1.0043, 1.0000, 1.0000, 1.0043,
        1.0043, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 102 was -0.5651809916833458
current params: tensor([3.0635, 3.0635, 3.0635, 3.0635, 3.0635, 1.0044, 1.0044, 1.0044, 3.0635,
        3.0635, 1.0044, 1.0044, 3.0635, 3.0635, 1.0044, 1.0000, 1.0000, 1.0044,
        1.0044, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10]

...

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 250 was -0.5418034357442743
current params: tensor([4.7077, 4.7077, 4.7077, 4.7077, 4.7077, 1.0106, 1.0106, 1.0106, 4.7077,
        4.7077, 1.0106, 1.0106, 4.7077, 4.7077, 1.0106, 1.0000, 1.0000, 1.0106,
        1.0106, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 251 was -0.5416545024976345
current params: tensor([4.7181, 4.7181, 4.7181, 4.7181, 4.7181, 1.0107, 1.0107, 1.0107, 4.7181,
        4.7181, 1.0107, 1.0107, 4.7181, 4.7181, 1.0107, 1.0000, 1.0000, 1.0107,
        1.0107, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 252 was -0.541506089957825
current params: tensor([4.7285, 4.7285, 4.7285, 4.7285, 4.7285, 1.0107, 1.0107, 1.0107, 4.7285,
        4.7285, 1.0107, 1.0107, 4.7285, 4.7285, 1.0107, 1.0000, 1.0000, 1.0107,
        1.0107, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10]

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 275 was -0.5382292413552865
current params: tensor([4.9670, 4.9670, 4.9670, 4.9670, 4.9670, 1.0117, 1.0117, 1.0117, 4.9670,
        4.9670, 1.0117, 1.0117, 4.9670, 4.9670, 1.0117, 1.0000, 1.0000, 1.0117,
        1.0117, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 276 was -0.5380924187354624
current params: tensor([4.9773, 4.9773, 4.9773, 4.9773, 4.9773, 1.0117, 1.0117, 1.0117, 4.9773,
        4.9773, 1.0117, 1.0117, 4.9773, 4.9773, 1.0117, 1.0000, 1.0000, 1.0117,
        1.0117, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 277 was -0.5379560474191649
current params: tensor([4.9876, 4.9876, 4.9876, 4.9876, 4.9876, 1.0118, 1.0118, 1.0118, 4.9876,
        4.9876, 1.0118, 1.0118, 4.9876, 4.9876, 1.0118, 1.0000, 1.0000, 1.0118,
        1.0118, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 300 was -0.5349375692191219
current params: tensor([5.2241, 5.2241, 5.2241, 5.2241, 5.2241, 1.0127, 1.0127, 1.0127, 5.2241,
        5.2241, 1.0127, 1.0127, 5.2241, 5.2241, 1.0127, 1.0000, 1.0000, 1.0127,
        1.0127, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 301 was -0.5348112365809968
current params: tensor([5.2344, 5.2344, 5.2344, 5.2344, 5.2344, 1.0128, 1.0128, 1.0128, 5.2344,
        5.2344, 1.0128, 1.0128, 5.2344, 5.2344, 1.0128, 1.0000, 1.0000, 1.0128,
        1.0128, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 302 was -0.5346852940056611
current params: tensor([5.2446, 5.2446, 5.2446, 5.2446, 5.2446, 1.0128, 1.0128, 1.0128, 5.2446,
        5.2446, 1.0128, 1.0128, 5.2446, 5.2446, 1.0128, 1.0000, 1.0000, 1.0128,
        1.0128, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 325 was -0.5318918669845479
current params: tensor([5.4797, 5.4797, 5.4797, 5.4797, 5.4797, 1.0138, 1.0138, 1.0138, 5.4797,
        5.4797, 1.0138, 1.0138, 5.4797, 5.4797, 1.0138, 1.0000, 1.0000, 1.0138,
        1.0138, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 326 was -0.5317747156696306
current params: tensor([5.4899, 5.4899, 5.4899, 5.4899, 5.4899, 1.0138, 1.0138, 1.0138, 5.4899,
        5.4899, 1.0138, 1.0138, 5.4899, 5.4899, 1.0138, 1.0000, 1.0000, 1.0138,
        1.0138, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 327 was -0.5316579088576402
current params: tensor([5.5000, 5.5000, 5.5000, 5.5000, 5.5000, 1.0139, 1.0139, 1.0139, 5.5000,
        5.5000, 1.0139, 1.0139, 5.5000, 5.5000, 1.0139, 1.0000, 1.0000, 1.0139,
        1.0139, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10

[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 350 was -0.5290625639319297
current params: tensor([5.7339, 5.7339, 5.7339, 5.7339, 5.7339, 1.0149, 1.0149, 1.0149, 5.7339,
        5.7339, 1.0149, 1.0149, 5.7339, 5.7339, 1.0149, 1.0000, 1.0000, 1.0149,
        1.0149, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 351 was -0.528953533777809
current params: tensor([5.7441, 5.7441, 5.7441, 5.7441, 5.7441, 1.0149, 1.0149, 1.0149, 5.7441,
        5.7441, 1.0149, 1.0149, 5.7441, 5.7441, 1.0149, 1.0000, 1.0000, 1.0149,
        1.0149, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10], [0]]
Optimizing Flux Correlations
yield on sim iteration 352 was -0.5288448087528856
current params: tensor([5.7542, 5.7542, 5.7542, 5.7542, 5.7542, 1.0149, 1.0149, 1.0149, 5.7542,
        5.7542, 1.0149, 1.0149, 5.7542, 5.7542, 1.0149, 1.0000, 1.0000, 1.0149,
        1.0149, 1.0000, 1.0000], dtype=torch.float64)
Using CPU
[[10]

In [ ]:
optim.optimize()

In [ ]:
# optim.optimize()

In [ ]:
# optim.optimize()

In [ ]:
def calc_var(v1,v2):
    sq_sum=0
    for i in range(len(v1)):
        sq_sum=(v1[i]-v2[i])**2+sq_sum
    
    sq_sum = ((sq_sum)**0.5)/(len(v1)-1)
    return(sq_sum)

In [ ]:
yields= []
final_params=[]
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i].item())
#     print(optim.final_solns[i].numpy())
    final_params.append(optim.final_solns[i].numpy())

sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)[sort_indx]
sorted_params = np.array(final_params)[sort_indx]

p0 = sorted_params[0]
var_params = []
for i in range(len(sorted_params)):
    var_params.append(calc_var(p0,sorted_params[i]))
    
arg_indx = np.argsort(np.array(var_params))
sorted_var = np.array(var_params)[arg_indx]

print(sorted_var[0])
print(sorted_var[-1])
print("Yield: ",sorted_yields[arg_indx[0]],"\nParams: ",sorted_params[arg_indx[0]])

print("Yield: ",sorted_yields[arg_indx[-1]],"\nParams: ",sorted_params[arg_indx[-1]])
print("Max Yield: ",sorted_yields[-1],"\nParams: ",sorted_params[-1])

In [ ]:
# optim.optimize()


In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off']}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (self._R * self._T)) + l_kon + torch.log(std_c)
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

pathway = []
kon_sumarray = []
total_con_rate = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]

print(pathway)
print(kon_sumarray)
#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','S','M','AB','BMS','ABS','AMS','ABMS','AM','AS']
#nodes_list = ['A','B','ABMS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:
optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1, 3)
optim_rn = optim.rn
for i, runtime in enumerate([1, 8, 64]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i],)
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
plt.show()

In [ ]:
node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 